In [8]:
import requests
import json 

request = requests.get("https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1\
    /odata/CotacaoDolarDia(dataCotacao=@dataCotacao)?%40dataCotacao='05-29-2025'&%24format=json")

response = request.json().get('value')

In [9]:
print(response)

[{'cotacaoCompra': 5.6553, 'cotacaoVenda': 5.6559, 'dataHoraCotacao': '2025-05-29 13:05:33.689'}]


In [ ]:
import pandas as pd
parquet_df = pd.read_parquet(r'D:\Projetos\Python\Data_Engineering\dolar-pipeline\data\dolar_1.parquet', engine='pyarrow')
parquet_df

,data_extracao


In [ ]:
import requests

url = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial='01-01-2024'&@dataFinalCotacao='05-15-2025'&$top=500&$format=json&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao"
response = requests.get(url)

if response.status_code == 200 and response.content:
    data = response.json().get('value', [])
    print(data)
else:
    print(f"Erro ou resposta vazia: status {response.status_code}")
    print(response.text)


[{'cotacaoCompra': 4.891, 'cotacaoVenda': 4.8916, 'dataHoraCotacao': '2024-01-02 13:05:50.319'}, {'cotacaoCompra': 4.9206, 'cotacaoVenda': 4.9212, 'dataHoraCotacao': '2024-01-03 13:14:41.153'}, {'cotacaoCompra': 4.9182, 'cotacaoVenda': 4.9188, 'dataHoraCotacao': '2024-01-04 13:05:35.048'}, {'cotacaoCompra': 4.8893, 'cotacaoVenda': 4.8899, 'dataHoraCotacao': '2024-01-05 13:10:31.843'}, {'cotacaoCompra': 4.8844, 'cotacaoVenda': 4.885, 'dataHoraCotacao': '2024-01-08 13:11:30.76'}, {'cotacaoCompra': 4.8931, 'cotacaoVenda': 4.8937, 'dataHoraCotacao': '2024-01-09 13:09:30.847'}, {'cotacaoCompra': 4.8895, 'cotacaoVenda': 4.8901, 'dataHoraCotacao': '2024-01-10 13:09:34.773'}, {'cotacaoCompra': 4.8788, 'cotacaoVenda': 4.8794, 'dataHoraCotacao': '2024-01-11 17:03:40.62'}, {'cotacaoCompra': 4.8537, 'cotacaoVenda': 4.8543, 'dataHoraCotacao': '2024-01-12 13:26:56.442'}, {'cotacaoCompra': 4.8759, 'cotacaoVenda': 4.8765, 'dataHoraCotacao': '2024-01-15 13:38:08.191'}, {'cotacaoCompra': 4.9032, 'cotaca

In [61]:
import pandas as pd
df = pd.DataFrame(data)
df.to_csv(r'D:\Projetos\Python\Data_Engineering\dolar-pipeline\data\dados.csv', index=False)

In [62]:
df['dataHoraCotacao'] = pd.to_datetime(df['dataHoraCotacao'])
df['data'] = df['dataHoraCotacao'].dt.date

In [63]:
df.head()

,cotacaoCompra,cotacaoVenda,dataHoraCotacao,data
0,4.8910,4.8916,2024-01-02 13:05:50.319,2024-01-02
1,4.9206,4.9212,2024-01-03 13:14:41.153,2024-01-03
2,4.9182,4.9188,2024-01-04 13:05:35.048,2024-01-04
3,4.8893,4.8899,2024-01-05 13:10:31.843,2024-01-05
4,4.8844,4.8850,2024-01-08 13:11:30.760,2024-01-08


In [64]:
datas = pd.date_range(start='2024-01-01', end='2025-05-15', freq='D')
df_datas = pd.DataFrame(datas, columns=['data'])

In [65]:
df['data'] = pd.to_datetime(df['data'])         # Isso transforma '2024-01-02' em datetime64
df_datas['data'] = pd.to_datetime(df_datas['data'])

In [ ]:
df_merged = pd.merge(df_datas, df[['data', 'cotacaoCompra', 'cotacaoVenda', 'dataHoraCotacao']], on='data', how='left')
df['data'] = pd.to_datetime(df['data'])
df_datas['data'] = pd.to_datetime(df_datas['data'])
df_merged.sort_values('data', inplace=True)
df_merged.fillna(method='bfill', inplace=True)  # preenche o primeiro valor
df_merged.fillna(method='ffill', inplace=True) 

C:\Users\Arthur Coutinho\AppData\Local\Temp\ipykernel_41552\1836214132.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged.fillna(method='ffill', inplace=True)


In [67]:
df_merged.to_csv(r'D:\Projetos\Python\Data_Engineering\dolar-pipeline\data\df_merged.csv', index=False)